In [1]:
import psycopg2
from psycopg2 import sql
import pandas as pd
import pickle
import datetime
import types
import pandas.io.sql as sqlio
from psycopg2 import sql

In [3]:
with open("../db_config.json", "r") as db_config:
    db_config = json.load(db_config)

# db 접속정보는 json으로 관리
dbname = db_config['local_db']['NAME']
host = db_config['local_db']['HOST'] 
port = db_config['local_db']['PORT'] 
user = db_config['local_db']['USER']
password= db_config['local_db']['PASSWORD'] 
connect_param_local = dict({'dbname':dbname, 'host':host, 'port':port, 'user':user, 'password':password})

# 1. 대상 회사 테이블 만들기

In [ ]:
with open('/Users/youngjinlim/OneDrive/Coding/BigData_Study/Final_project/Data/Company_list/company_list_df.pkl', 'rb') as f:
    company_list = pickle.load(f)

In [ ]:
# 대상 회사 테이블 만들기
with psycopg2.connect(**connect_param_local) as con:
    with con.cursor() as cur:
        cur.execute('drop table if exists target_company_list')
        create_companylist_sql = '''
        create table target_company_list(
        stock_code char(6) primary key
        ,company_name varchar(50) NOT NULL
        ,market char(1) NOT NULL
        ,bankruptcy char(1) NOT NULL
        ,delisted_check char(1) NOT NULL
        ,delisted_date date NOT NULL
        ,pre_3m date NOT NULL
        ,pre_6m date NOT NULL
        ,pre_1y date NOT NULL
        );
        '''
        cur.execute(create_companylist_sql)
        con.commit()

In [ ]:
# 대상 회사 테이블에 값 넣기
with psycopg2.connect(**connect_param_local) as con:
    with con.cursor() as cur:
        insert_companyvalue_sql = 'insert into target_company_list values (%s, %s, %s, %s, %s, %s, %s, %s, %s);'
        for each_idx in company_list.index:
            tuple_inserted_listtype = []
            for each_value in company_list.iloc[each_idx].values:
                if str(type(each_value)) == "<class 'numpy.int64'>":
                    tuple_inserted_listtype.append(int(each_value))
                else:
                    tuple_inserted_listtype.append(each_value)
            cur.execute(insert_companyvalue_sql, tuple(tuple_inserted_listtype))
            con.commit()

# 2. 주식 데이터 테이블 만들기

## 2-1. 주식 데이터 테이블 원본 만들기

In [ ]:
# target_company_list에 없는 종목 제거. 딱 하나였음
df_cache = pd.read_csv('stock_data_raw_transposed.csv', chunksize=100000, dtype='object')
head_check = 1
for each_chunk in df_cache:
    df_cache2 = each_chunk[each_chunk['stock_code'] != '042950']
    if head_check == 1:
        df_cache2.to_csv('stock_data_raw_transposed.csv', mode='a', index=False)
        head_check += 1
    else:
        df_cache2.to_csv('stock_data_raw_transposed.csv', mode='a', index=False, header=False)

In [ ]:
# 주식 데이터 테이블 만들기
get_header = next(pd.read_csv('stock_data_raw.csv', chunksize=1)).columns
with psycopg2.connect(**connect_param_local) as con:
    with con.cursor() as cur:
        cur.execute('drop table if exists stock_data_2000_2020_raw')
        
        create_stocktable_sql = '''
        create table stock_data_2000_2020_raw(
        company_name varchar(50) NOT NULL
        ,date date NOT NULL
        ,stock_code char(6) NOT NULL
        ,FOREIGN KEY (stock_code) REFERENCES target_company_list(stock_code));
        '''
        cur.execute(create_stocktable_sql)
        
        prefix_add_columns = 'ALTER TABLE stock_data_2000_2020_raw ADD COLUMN'
        for each_column in get_header[3:]:
            add_columns_to_table = prefix_add_columns + ' ' + '"' + each_column + '"' + ' double precision'
            cur.execute(add_columns_to_table)
        
        con.commit()

In [ ]:
# csv copy. 내용 채워넣기
with psycopg2.connect(**connect_param_local) as con:
    with con.cursor() as cur:
        cur.execute(
            """
            COPY stock_data_2000_2020_raw
            from '/usr/local/var/postgres/stock_data_raw.csv'
            DELIMITER ','
            CSV HEADER;
            """
        )
        con.commit()

## 2-2. 주가 데이터 3년치만 담고 있는 테이블 만들기
* 3개월 전 예측
* 6개월 전 예측
* 1년 전 예측

In [3]:
def create_finance_table_for_ml(prediction_range:'6m', years:3, timedelta:1096):
    get_header = next(pd.read_csv('csv_data/stock_data_raw.csv', chunksize=1)).columns
    
    with psycopg2.connect(**connect_param_local) as con:
        with con.cursor() as cur:
            table_name_argu = 'stock_data_' + str(years) + 'years_raw_' + prediction_range
            drop_table_query = sql.SQL('drop table if exists {table_name}').format(table_name=sql.Identifier(table_name_argu))
            cur.execute(drop_table_query)
            
            create_stocktable_sql = """
            create table {table_name} (
            ,company_name varchar(50) NOT NULL
            ,date date NOT NULL
            ,stock_code char(6) NOT NULL
            ,FOREIGN KEY (stock_code) REFERENCES target_company_list(stock_code));
            """.format(table_name = sql.Identifier(table_name = sql.Identifier(table_name_argu))
            cur.execute(create_stocktable_sql)
            
            con.commit()

            # 테이블에 열 추가하기
            prefix_add_columns = sql.SQL('ALTER TABLE {table_name} ADD COLUMN').format(table_name = sql.Identifier(table_name = sql.Identifier(table_name_argu))
            for each_column in get_header[3:]:
                add_columns_to_table = prefix_add_columns + ' ' + '"' + each_column + '"' + ' double precision'
                print(add_columns_to_table)
                cur.execute(add_columns_to_table)
            con.commit()
                                                                                       
                                                                                       
    with psycopg2.connect(**connect_param_local) as con:
        cur_2 = con.cursor()
        # fetchall을 사용해서 커서를 재활용하지 않고 커서를 두 개 두는 이유는, redshift의 single-node cluster에서는 fetchall이 지원되지 않기 떄문이다.
            # InternalError_: Fetch ALL is not supported on single-node clusters.
            # Please specify the fetch size (maximum 1000 for single-node clusters) 
            # or upgrade to a multi node installation.

        get_companylist_sql = """select * from target_company_list;"""
        target_company_list = sqlio.read_sql_query(get_companylist_sql, con)
        # 1mb도 안되는 작은 테이블이라서 데이터프레임으로 한 번에 받아옴

        for each_code in target_company_list['stock_code']:
            cur_1 = con.cursor('ss_cursor') # server side cursor
            # cur_1.itersize = 1000 # redshift single-node cluster에서의 server side cursor의 최대 제한값

            print(each_code,'_start')
            predict_start = target_company_list[target_company_list['stock_code'] == each_code]['pre_' + prediction_range].values[0] - datetime.timedelta(days=1)
            predict_end = predict_start - datetime.timedelta(days = timedelta)

            cur_1.execute(
                """
                select * from stock_data_2000_2020_raw
                where (date between %(predict_end)s and %(predict_start)s) and (stock_code = %(stock_code)s);
                """,
                {'predict_end':predict_end.strftime("%Y-%m-%d"),'predict_start':predict_start.strftime("%Y-%m-%d"),'stock_code': each_code}
            )

            for each_row in cur_1:
                # fetchall을 못 써서 cur_2가 등장하는 부분
                # next(cur_1)로 한줄씩 불러와서 insert
                query_insert = sql.SQL('insert into {table_name} values %s').format(table_name = sql.Identifier(table_name_argu))
                cur_2.execute(query_insert, [each_row]) # each_row는 tuple이다.

            con.commit()
            print(each_code,'_commit complete')
        cur_2.close()

# 3. 감사보고서 데이터 테이블 만들기

In [7]:
with psycopg2.connect(**connect_param_local) as con:
    with con.cursor() as cur:
        # 감사보고서 테이블 만들기
        cur.execute('drop table if exists dart_audit_report_data')
        create_stocktable_sql = """
        create table dart_audit_report_data(
        stock_code char(6)
        ,FOREIGN KEY (stock_code) REFERENCES target_company_list(stock_code)
        ,company_name varchar(50) NOT NULL
        ,report_name char(4)
        ,n_opinion boolean
        ,report_date date);
        """
        cur.execute(create_stocktable_sql)
        con.commit()

In [26]:
# csv copy
with psycopg2.connect(**connect_param_local) as con:
    with con.cursor() as cur:
        cur.execute(
            """
            COPY dart_audit_report_data
            from '/usr/local/var/postgres/dart_audit_report_data_refined.csv'
            DELIMITER ','
            CSV HEADER;
            """
        )
        con.commit()

# 4. 재무 데이터 테이블 만들기

## 4-1. 재무 데이터 전체 원본 테이블 만들기

In [86]:
df_cache = pd.read_csv('/Users/youngjinlim/OneDrive/Coding/BigData_Study/Final_project/AWS/COPY용 데이터/finance_data_1999_2020_raw.csv', dtype={'stock_code':object}, chunksize=1)
get_header_finance = next(df_cache).columns

with psycopg2.connect(**connect_param_local) as con:
    with con.cursor() as cur:
        # 재무 데이터 테이블 만들기
        cur.execute('drop table if exists finance_data_1999_2020_raw')
        create_stocktable_sql = '''
        create table finance_data_1999_2020_raw(
        stock_code char(6) NOT NULL
        ,FOREIGN KEY (stock_code) REFERENCES target_company_list(stock_code)
        ,company_name varchar(50) NOT NULL
        ,결산월 smallint
        ,회계년 smallint
        );
        '''
        cur.execute(create_stocktable_sql)
        con.commit()
        
        # 테이블에 열 추가하기
        prefix_add_columns = 'ALTER TABLE finance_data_1999_2020_raw ADD COLUMN'
        for each_column in get_header_finance[4:]:
            if each_column in ['부채비율(p)', '자기자본비율(p)' ,'당기순이익증가율(전년동기)(p)', 'EPS증가율(전년동기)(p)', 'ROE(당기순이익)(p)', '매출액증가율(전년동기)(p)']:
                add_columns_to_table = prefix_add_columns + ' ' + '"' + each_column + '"' + ' varchar(15)' 
                    # 계산불가능한 항목(divided by 0 등)은 별도의 텍스트로 표시되어있기 때문에 varchar 선택 
            else:
                add_columns_to_table = prefix_add_columns + ' ' + '"' + each_column + '"' + ' double precision'
            cur.execute(add_columns_to_table)
            print(each_column)
        con.commit()

총자산(천원)
유동자산(천원)
현금및현금성자산(천원)
재고자산(천원)
비유동자산(천원)
총부채(천원)
유동부채(천원)
비유동부채(천원)
총자본(천원)
매출액(천원)
당기순이익(천원)
세전계속사업이익(천원)
영업이익(천원)
이자비용(천원)
비지배주주지분(천원)
계속사업법인세비용(천원)
매출총이익(천원)
금융원가(비영업)(천원)
부채비율(p)
자기자본비율(p)
차입금의존도(p)
당기순이익률(p)
세전계속사업이익률(p)
영업이익률(p)
매출총이익률(p)
ROE(당기순이익)(p)
총자산회전율(회)
매출액증가율(전년동기)(p)
총자산증가율(전년동기)(p)
당기순이익증가율(전년동기)(p)
EPS증가율(전년동기)(p)


In [87]:
# csv copy
with psycopg2.connect(**connect_param_local) as con:
    with con.cursor() as cur:
        cur.execute(
            """
            COPY finance_data_1999_2020_raw
            from '/usr/local/var/postgres/finance_data_1999_2020_raw.csv'
            DELIMITER ','
            CSV HEADER;
            """
        )
        con.commit()

In [89]:
# 컬럼 두 개 추가
with psycopg2.connect(**connect_param_local) as con:
    with con.cursor() as cur:
        cur.execute('ALTER TABLE finance_data_1999_2020_raw ADD COLUMN "사업보고서마감일" date')
        cur.execute(
            """
            UPDATE
            finance_data_1999_2020_raw
            SET
            "사업보고서마감일" = (date_trunc('month',  ("회계년" || '-' || "결산월" || '-' || 10)::date) + interval '1 month' - interval '1 day')::date+90;
            """
        ) # 각 월의 최종일에서 90일을 더한 값을 저장하는 열을 추가한다. month 단위로 계산되기 때문에, 결산월 뒤의 일은 아무거나 가져다 붙여도 된다. 여기서는 10일을 넣었음
        # 결산월 90일 이내에는 사업보고서 공시를 올려야 하기 때문에, 결산월 90일 후에는 무조건 데이터를 볼 수 있을 것이라고 가정한 것이다.
        cur.execute('ALTER TABLE finance_data_1999_2020_raw ADD COLUMN "총자산부채비율(p)" double precision')
        cur.execute(
            """
            UPDATE
            finance_data_1999_2020_raw
            SET
            "총자산부채비율(p)" = "총부채(천원)"/"총자산(천원)";
            """
        )
        con.commit()

In [9]:
# 인덱스 생성
with psycopg2.connect(**connect_param_local) as con:
    with con.cursor() as cur:
        cur.execute(
            """
            CREATE INDEX "idx_(stock_code+사업보고서마감일)_finance_data_1999_2020_raw"
            ON finance_data_1999_2020_raw (stock_code, "사업보고서마감일");
            """
        )
        con.commit()

## 4-2. 재무 데이터 머신러닝 학습용 2년치, 3년치 테이블 만들기
* 3개월 후 예측
* 6개월 후 예측
* 1년 후 예측

In [12]:
step_1_column_finance_3 = [
    'Y-1_총자산(천원)'
    ,'Y-1_현금및현금성자산(천원)'
    ,'Y-1_총부채(천원)'
    ,'Y-1_총자본(천원)'
    ,'Y-1_매출액(천원)'
    ,'Y-1_당기순이익(천원)'
    ,'Y-1_세전계속사업이익(천원)'
    ,'Y-1_영업이익(천원)'
    ,'Y-1_매출총이익(천원)'
    ,'Y-1_차입금의존도(p)'
    ,'Y-1_당기순이익률(p)'
    ,'Y-1_세전계속사업이익률(p)'
    ,'Y-1_영업이익률(p)'
    ,'Y-1_매출총이익률(p)'
    ,'Y-1_총자산회전율(회)'
    ,'Y-1_총자산증가율(p)'
    ,'Y-1_총자산부채비율(p)'
    ,'Y-2_총자산(천원)'
    ,'Y-2_현금및현금성자산(천원)'
    ,'Y-2_총부채(천원)'
    ,'Y-2_총자본(천원)'
    ,'Y-2_매출액(천원)'
    ,'Y-2_당기순이익(천원)'
    ,'Y-2_세전계속사업이익(천원)'
    ,'Y-2_영업이익(천원)'
    ,'Y-2_매출총이익(천원)'
    ,'Y-2_차입금의존도(p)'
    ,'Y-2_당기순이익률(p)'
    ,'Y-2_세전계속사업이익률(p)'
    ,'Y-2_영업이익률(p)'
    ,'Y-2_매출총이익률(p)'
    ,'Y-2_총자산회전율(회)'
    ,'Y-2_총자산증가율(p)'
    ,'Y-2_총자산부채비율(p)'
    ,'Y-3_총자산(천원)'
    ,'Y-3_현금및현금성자산(천원)'
    ,'Y-3_총부채(천원)'
    ,'Y-3_총자본(천원)'
    ,'Y-3_매출액(천원)'
    ,'Y-3_당기순이익(천원)'
    ,'Y-3_세전계속사업이익(천원)'
    ,'Y-3_영업이익(천원)'
    ,'Y-3_매출총이익(천원)'
    ,'Y-3_차입금의존도(p)'
    ,'Y-3_당기순이익률(p)'
    ,'Y-3_세전계속사업이익률(p)'
    ,'Y-3_영업이익률(p)'
    ,'Y-3_매출총이익률(p)'
    ,'Y-3_총자산회전율(회)'
    ,'Y-3_총자산증가율(p)'
    ,'Y-3_총자산부채비율(p)'
]

step_1_column_finance_2 = [
    'Y-1_총자산(천원)'
    ,'Y-1_현금및현금성자산(천원)'
    ,'Y-1_총부채(천원)'
    ,'Y-1_총자본(천원)'
    ,'Y-1_매출액(천원)'
    ,'Y-1_당기순이익(천원)'
    ,'Y-1_세전계속사업이익(천원)'
    ,'Y-1_영업이익(천원)'
    ,'Y-1_매출총이익(천원)'
    ,'Y-1_차입금의존도(p)'
    ,'Y-1_당기순이익률(p)'
    ,'Y-1_세전계속사업이익률(p)'
    ,'Y-1_영업이익률(p)'
    ,'Y-1_매출총이익률(p)'
    ,'Y-1_총자산회전율(회)'
    ,'Y-1_총자산증가율(p)'
    ,'Y-1_총자산부채비율(p)'
    ,'Y-2_총자산(천원)'
    ,'Y-2_현금및현금성자산(천원)'
    ,'Y-2_총부채(천원)'
    ,'Y-2_총자본(천원)'
    ,'Y-2_매출액(천원)'
    ,'Y-2_당기순이익(천원)'
    ,'Y-2_세전계속사업이익(천원)'
    ,'Y-2_영업이익(천원)'
    ,'Y-2_매출총이익(천원)'
    ,'Y-2_차입금의존도(p)'
    ,'Y-2_당기순이익률(p)'
    ,'Y-2_세전계속사업이익률(p)'
    ,'Y-2_영업이익률(p)'
    ,'Y-2_매출총이익률(p)'
    ,'Y-2_총자산회전율(회)'
    ,'Y-2_총자산증가율(p)'
    ,'Y-2_총자산부채비율(p)'
]

extracted_value_finance= [
    '총자산(천원)'
    ,'현금및현금성자산(천원)'
    ,'총부채(천원)'
    ,'총자본(천원)'
    ,'매출액(천원)'
    ,'당기순이익(천원)'
    ,'세전계속사업이익(천원)'
    ,'영업이익(천원)'
    ,'매출총이익(천원)'
    ,'차입금의존도(p)'
    ,'당기순이익률(p)'
    ,'세전계속사업이익률(p)'
    ,'영업이익률(p)'
    ,'매출총이익률(p)'
    ,'총자산회전율(회)'
    ,'총자산증가율(전년동기)(p)'
    ,'총자산부채비율(p)'  
]

In [27]:
def create_finance_table_for_ml(prediction_range:'6m', years:2, timedelta:730):
    
    table_name_argument = 'table_for_dt_based_model_' + prediction_range + '_finance_' + str(years) + 'y'
    if years == 3:
        step_1_column_finance = step_1_column_finance_3
    elif years == 2:
        step_1_column_finance = step_1_column_finance_2
    
    with psycopg2.connect(**connect_param_local) as con:
        with con.cursor() as cur:
            # 재무 데이터 테이블 만들기
            query_drop_table = sql.SQL('drop table if exists {table_name}').format(table_name = sql.Identifier(table_name_argument))
            cur.execute(query_drop_table)
            create_stocktable_sql = sql.SQL('''
            create table {table_name}(
            stock_code char(6) NOT NULL,
            FOREIGN KEY (stock_code) REFERENCES target_company_list(stock_code)
            );
            ''').format(table_name = sql.Identifier(table_name_argument))
            cur.execute(create_stocktable_sql)
            con.commit()

            # 테이블에 열 추가하기
            for each_column in step_1_column_finance:
                query_prefix_add_columns = sql.SQL('ALTER TABLE {table_name} ADD COLUMN {column} double precision').format(table_name = sql.Identifier(table_name_argument), column = sql.Identifier(each_column))
                cur.execute(query_prefix_add_columns)
                print(each_column)
            con.commit()


    i = 1
    insert_values_query_setting = '%s'
    while (i <= len(step_1_column_finance)):
        insert_values_query_setting = insert_values_query_setting + ',%s'
        i+=1
    print(insert_values_query_setting)
    
    
    with psycopg2.connect(**connect_param_local) as con:
        with con.cursor() as cur:
            get_stock_code_query = """
            SELECT DISTINCT stock_code from finance_data_1999_2020_raw;
            """

            get_date_query = sql.SQL(
                """
                select {pre_date_column} 
                from target_company_list 
                where stock_code = %(stock_code)s;
                """
            ).format(pre_date_column = sql.Identifier('pre_' + prediction_range))
            
            create_cache_query = """
            create view finance_cache
            as (
            select *,
            row_number() over(order by "회계년" desc) as rn
            from finance_data_1999_2020_raw
            where (stock_code = %(stock_code)s) AND ("사업보고서마감일" < %(pre_date)s)
            );
            """

            cur.execute(get_stock_code_query)
            stock_codes = cur.fetchall()

            for each_code in stock_codes:
                cur.execute(get_date_query, {'stock_code':each_code[0]})
                pre_date_extracted = cur.fetchone()[0].strftime("%Y-%m-%d")
                drop_cache_query = "drop view if exists finance_cache;"
                cur.execute(drop_cache_query)
                con.commit()
                cur.execute(create_cache_query, {'stock_code':each_code[0], 'pre_date':pre_date_extracted})
                con.commit()

                insert_list = []
                insert_list.append(each_code[0])
                for y_num in range(1,years+1): # 최신 year년치만
                    for each_value in extracted_value_finance:
                        select_target_value_query = sql.SQL("""
                        select {target_value}
                        from finance_cache
                        where rn = %(y_num)s;""").format(target_value = sql.Identifier(each_value))
                        cur.execute(select_target_value_query, {'y_num':y_num})
                        insert_value = cur.fetchone()
                        if insert_value == None:
                            insert_list.append(None)
                        else:
                            # print(insert_value[0])
                            insert_list.append(insert_value[0])

                insert_query_cache = 'insert into {table_name} values' + '(' + insert_values_query_setting + ')'
                insert_query = sql.SQL(insert_query_cache).format(table_name = sql.Identifier(table_name_argument))
                cur.execute(insert_query,tuple(insert_list))
                con.commit()
                print(each_code)
                
    with psycopg2.connect(**connect_param_local) as con:
        with con.cursor() as cur:
            # 한 컬럼이라도 NULL이 있는 ROW 삭제
            query_delete_null_rows = sql.SQL("""
                delete from {table_name}
                where not ({table_name} is not null);
            """).format(table_name = sql.Identifier(table_name_argument))
            cur.execute(query_delete_null_rows)
            con.commit()

In [ ]:
create_finance_table_for_ml(prediction_range='6m', years=2, timedelta=730)
create_finance_table_for_ml(prediction_range='6m', years=3, timedelta=1096)
create_finance_table_for_ml(prediction_range='3m', years=2, timedelta=730)
create_finance_table_for_ml(prediction_range='3m', years=3, timedelta=1096)
create_finance_table_for_ml(prediction_range='1y', years=2, timedelta=730)
create_finance_table_for_ml(prediction_range='1y', years=3, timedelta=1096)

# 5. 뉴스 데이터 테이블 만들기

In [27]:
# 뉴스 데이터 메인 테이블 만들기
# 중복제거된 1224734개의 기사가 들어가게 됨
with psycopg2.connect(**connect_param_local) as con:
    with con.cursor() as cur:
        cur.execute('drop table if exists news_data_deduplicated_main CASCADE')
        con.commit()
        cur.execute(
            """
            CREATE TABLE news_data_deduplicated_main(
                article_url text NOT NULL PRIMARY KEY
                ,published_date date NOT NULL
                ,article_title text
                ,article_body text
                ,bankruptcy_article_check boolean);
            """
        )
        con.commit()
        cur.execute(
            """
            COPY news_data_deduplicated_main
            from '/usr/local/var/postgres/news_data_deduplicated_main.csv'
            DELIMITER ','
            CSV HEADER;
            """
        )
        con.commit()

In [29]:
# 뉴스 데이터 URL 테이블 만들기
# 중복제거 되어있지 않음. 하나의 기사가 여러 기업에 연관되어있는 1:N의 관계이기 때문에, 이를 표현해 줄 이 테이블이 필요함
# 총 2380889행
with psycopg2.connect(**connect_param_local) as con:
    with con.cursor() as cur:
        cur.execute('drop table if exists news_data_total_url_for_tag')
        con.commit()
        cur.execute(
            """
            CREATE TABLE news_data_total_url_for_tag(
                stock_code char(6) NOT NULL
                ,FOREIGN KEY (stock_code) REFERENCES target_company_list(stock_code)
                ,company_name varchar(50) NOT NULL
                ,article_url text NOT NULL
                ,FOREIGN KEY (article_url) REFERENCES news_data_deduplicated_main(article_url));
            """
        )
        con.commit()
        cur.execute(
            """
            COPY news_data_total_url_for_tag
            from '/usr/local/var/postgres/news_data_total_url_for_tag.csv'
            DELIMITER ','
            CSV HEADER;
            """
        )
        con.commit()